# Introducción a datos Ráster (GDAL)

En esta sección, mostraremos el uso de la librería GDAL para trabajar con datos geoespaciales en general y con productos de datos de radar generados con el software ISCE. Comenzaremos con los conceptos básicos de las representaciones de datos geoespaciales y trabajaremos a través de varias operaciones comunes que involucran imágenes geoespaciales.

Al final de esta sección, utilizaremos GDAL para manipular productos de interferogramas estándar sin envolver del archivo ARIA/GRFN.

## Configuración

In [4]:
# Ejecute siempre esta celda al comienzo de cada sesión
import os
from osgeo import gdal, osr
from pathlib import Path
import zipfile

if 'nb_dir' not in globals():
    nb_dir = os.getcwd()

## Descargar conjuntos de datos para notebook

Descargaremos los conjuntos de datos utilizados en este cuaderno.

### Datos 1: mosaico SRTM DEM en formato geotiff sobre California de topografía abierta

In [5]:
!curl -X GET "https://portal.opentopography.org/API/globaldem?demtype=SRTMGL1_E&south=33&north=34&west=-119&east=-118&outputFormat=GTiff" -H "accept: */*" --output N33W119_wgs84.tif

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   121    0   121    0     0    114      0 --:--:--  0:00:01 --:--:--   114


### Datos 2: DEM de Groenlandia en formato netcdf de la Universidad de Montana

In [6]:
!wget -nc http://websrv.cs.umt.edu/isis/images/a/ab/Greenland1km.nc

--2022-09-07 10:58:36--  http://websrv.cs.umt.edu/isis/images/a/ab/Greenland1km.nc
Resolviendo websrv.cs.umt.edu (websrv.cs.umt.edu)... 150.131.192.24
Conectando con websrv.cs.umt.edu (websrv.cs.umt.edu)[150.131.192.24]:80... conectado.
Petición HTTP enviada, esperando respuesta... 200 OK
Longitud: 84110176 (80M) [application/x-netcdf]
Grabando a: «Greenland1km.nc»

Greenland1km.nc     100%[===================>]  80.21M  4.01MB/s    en 17s     

2022-09-07 10:58:54 (4.83 MB/s) - «Greenland1km.nc» guardado [84110176/84110176]



### Datos 3: DEM del sur de California en formato GMT de UC Santa Barbara

In [7]:
!wget -nc https://projects.eri.ucsb.edu/mapcat/mapftp/18.grd

--2022-09-07 11:00:52--  https://projects.eri.ucsb.edu/mapcat/mapftp/18.grd
Resolviendo projects.eri.ucsb.edu (projects.eri.ucsb.edu)... 128.111.100.88
Conectando con projects.eri.ucsb.edu (projects.eri.ucsb.edu)[128.111.100.88]:443... conectado.
Petición HTTP enviada, esperando respuesta... 200 OK
Longitud: 2888672 (2.8M)
Grabando a: «18.grd»

18.grd              100%[===================>]   2.75M   680KB/s    en 4.4s    

2022-09-07 11:00:58 (637 KB/s) - «18.grd» guardado [2888672/2888672]



### Datos 4: USGS DEM sobre Bushkill, PA desde PSU

In [8]:
!wget -nc https://courseware.e-education.psu.edu/downloads/natureofgeoinfo/DEM.zip

--2022-09-07 11:01:21--  https://courseware.e-education.psu.edu/downloads/natureofgeoinfo/DEM.zip
Resolviendo courseware.e-education.psu.edu (courseware.e-education.psu.edu)... 146.186.26.53
Conectando con courseware.e-education.psu.edu (courseware.e-education.psu.edu)[146.186.26.53]:443... conectado.
Petición HTTP enviada, esperando respuesta... 200 OK
Longitud: 2600801 (2.5M) [application/zip]
Grabando a: «DEM.zip»

DEM.zip             100%[===================>]   2.48M   943KB/s    en 2.7s    

2022-09-07 11:01:25 (943 KB/s) - «DEM.zip» guardado [2600801/2600801]



### Datos 5: datos en formato ISCE del procesamiento stripmapApp.py de Hawái

In [10]:
# descargue y descomprima algunos datos preprocesados de un depósito de AWS

def asf_unzip(output_dir, file_path):
    """
    Toma una ruta de directorio de salida y una ruta de archivo a un archivo comprimido.
    Si el archivo es un zip válido, extrae todo en el directorio de salida.
    """
    ext = file_path.suffix
    assert ext == '.zip', 'Error: file_path debe ser la ruta de un zip'

    if output_dir.is_dir() and file_path.is_file():
        print(f"Extrayendo: {file_path}")
        try:
            zipfile.ZipFile(file_path).extractall(output_dir)
        except zipfile.BadZipFile:
            print(f"Error de archivo comprimido.")
        return
    
path = "./stripmap.zip"
!aws --region=us-east-1 --no-sign-request s3 cp s3://asf-jupyter-data/unavco2021/stripmap.zip $path

path = Path(path)
asf_unzip(path.parent, path)
path.unlink()

download: s3://asf-jupyter-data/unavco2021/stripmap.zip to ./stripmap.zip
Extrayendo: stripmap.zip


### Datos 6: DEM en formato ISCE de Copernicus DEM
Aquí usamos una nueva utilidad de descarga DEM desarrollada por Scott Staniewicz en JPL llamada 'sardem' que ha sido preinstalada en los servidores 'unavco2022'. El siguiente comando descargará una sola celda de grado 1x1 de Copernicus GLO30 DEM, la convertirá a alturas elipsoidales y creará los archivos de metadatos ISCE2.

Consulte https://github.com/scottstanie/sardem para instalarlo en otra computadora.

In [13]:
#!sardem --bbox -119 34 -118 35 --data COP -isce
!sardem --bbox -119 34 -118 35 -isce
# no especificamos el nombre del archivo de salida, por lo que creó "elevation.dem"

[09/07 12:59:40] [INFO dem.py] Bounds: -119.0 34.0 -118.0 35.0
[09/07 12:59:40] [INFO download.py] Downloading https://e4ftl01.cr.usgs.gov/MEASURES/SRTMGL1.003/2000.02.11/N34W119.SRTMGL1.hgt.zip
[09/07 12:59:40] [INFO download.py] Using netrc file: /Users/javjrg/.netrc
[09/07 13:00:03] [INFO download.py] Writing to /Users/javjrg/.cache/sardem/N34W119.hgt.zip
[09/07 13:00:03] [INFO download.py] Unzipping /Users/javjrg/.cache/sardem/N34W119.hgt.zip
Archive:  /Users/javjrg/.cache/sardem/N34W119.hgt.zip
  inflating: /Users/javjrg/.cache/sardem/N34W119.hgt  
[09/07 13:00:04] [INFO dem.py] Cropping stitched DEM to boundaries
[09/07 13:00:05] [INFO dem.py] Rate = 1: No upsampling to do
[09/07 13:00:05] [INFO dem.py] Writing DEM to elevation.dem
[09/07 13:00:05] [INFO dem.py] Writing .dem.rsc file to elevation.dem.rsc
[09/07 13:00:05] [INFO dem.py] Creating ISCE2 XML file
This is the Open Source version of ISCE.
Some of the workflows depend on a separate licensed package.
To obtain the license

## GIS - Sistema de información geográfica (Geographic Information System)

GIS es un marco para organizar, administrar, visualizar y analizar datos geoespaciales. Como geofísicos, geólogos o geodestas, todos usamos este conjunto de herramientas para trabajar con datos en mapas. Todos los datos que se pueden trazar en un mapa se pueden clasificar como datos **geoespaciales**.

Las herramientas GIS gratuitas más populares incluyen:

1. Generic Mapping Tools (GMT) http://gmt.soest.hawaii.edu/
2. Geospatial Data Abstraction Library (GDAL) http://www.gdal.org/
3. QGIS https://qgis.org/
4. PROJ.4 / pyproj https://proj4.org/
5. Otras como Shapely, Fiona, Caropy, Basemap etc 

En esta sección, nos centraremos en la <u>Geospatial Data Abstraction Library (GDAL)</u>.

### Tipos de datos geoespaciales

Los datos geoespaciales generalmente se pueden clasificar en 2 tipos

#### Datos ráster

Desde la página web de ARCGIS:

En su forma más simple, un ráster consta de una matriz de celdas (o píxeles) organizadas en filas y columnas (o una cuadrícula) donde cada celda contiene un valor que representa información. Los ejemplos típicos de imágenes de trama incluyen:

1. Mapas de radares meteorológicos
2. Imágenes de satélite

Claramente, a partir de los ejercicios que hemos estado haciendo con los datos SAR, estos tipos de imágenes pertenecen a conjuntos de datos ráster. Este tutorial se centrará en la manipulación de datos ráster.

#### Datos vectoriales

Los datos vectoriales se refieren a conjuntos de datos que intentan representar información utilizando construcciones geométricas como puntos, líneas y polígonos. Los ejemplos típicos de datos vectoriales incluyen:

1. Carreteras, límites de países, etc.
2. Ubicaciones de terremotos de un catálogo
3. Características en forma de polígono como costas, etc.

Con SAR/InSAR, estos tipos de conjuntos de datos tienden a usarse de manera auxiliar para ayudar en la interpretación. Este tipo de datos no se discutirá en detalle como tal en este tutorial.

### ¿Por qué GDAL?

* Gratis y de código abierto (https://github.com/OSGeo/gdal)
* Soporte para más de 80 formatos de imagen y proyecciones de mapas
* Línea de comandos y API C/C++/Python/R
* Utilizado ampliamente por los grandes servicios de datos geoespaciales del mundo
* Amplio conjunto de pruebas y comunidad activa de desarrolladores
* GDAL también incluye una amplia compatibilidad con conjuntos de datos vectoriales (no es un tema de este tutorial)

<b><font color="blue">Nota:<br>
     En esta sección se debe usar específicamente con GDAL 3.x
     </font></b>

## Modelo de datos ráster en GDAL

GDAL tiene un modelo de datos muy claramente definido. Para obtener una descripción detallada de todos los aspectos del modelo de datos, consulte http://www.gdal.org/gdal_datamodel.html. En este tutorial, nos centraremos en algunos aspectos importantes de este modelo.

### Bandas de ráster

Una imagen ráster compatible con GDAL puede constar de muchas matrices 2D contenidas en ella, siempre que

1. Son del mismo tamaño
2. Representan información de los mismos puntos, es decir, las coordenadas del mapa son las mismas.
3. Algunos formatos de datos inherentemente permiten que las diferentes bandas sean de diferentes tipos de datos, pero esto no es muy común.

Veamos ejemplos de imágenes de banda única y multibanda usando la utilidad **gdalinfo**.

In [18]:
# Veamos un mosaico SRTM descargado
# Este es un archivo de una sola banda

!gdalinfo N33W119_wgs84.tif

Driver: GTiff/GeoTIFF
Files: N33W119_wgs84.tif
Size is 3600, 3600
Coordinate System is:
GEOGCRS["WGS 84",
    DATUM["World Geodetic System 1984",
        ELLIPSOID["WGS 84",6378137,298.257223563,
            LENGTHUNIT["metre",1]]],
    PRIMEM["Greenwich",0,
        ANGLEUNIT["degree",0.0174532925199433]],
    CS[ellipsoidal,2],
        AXIS["geodetic latitude (Lat)",north,
            ORDER[1],
            ANGLEUNIT["degree",0.0174532925199433]],
        AXIS["geodetic longitude (Lon)",east,
            ORDER[2],
            ANGLEUNIT["degree",0.0174532925199433]],
    ID["EPSG",4326]]
Data axis to CRS axis mapping: 2,1
Origin = (-119.000138888880798,34.000138888885445)
Pixel Size = (0.000277777777778,-0.000277777777778)
Metadata:
  AREA_OR_POINT=Area
Image Structure Metadata:
  COMPRESSION=LZW
  INTERLEAVE=BAND
  LAYOUT=COG
Corner Coordinates:
Upper Left  (-119.0001389,  34.0001389) (119d 0' 0.50"W, 34d 0' 0.50"N)
Lower Left  (-119.0001389,  33.0001389) (119d 0' 0.50"W, 33d 0' 0.50"N

<div class="alert alert-danger">
<b>PROBLEMA POTENCIAL:</b>
Si ve `ERROR 4: 'N33W119_wgs84.tif' not recognized as a supported file format.` arriba, entonces la descarga de OpenTopography falló. A partir de enero de 2022, requieren una clave API de OpenTopography para todas las solicitudes de descarga.

Consulte https://portal.opentopography.org/lidarAuthorizationInfo para obtener más información sobre cómo obtener la clave API. Cuando tenga la clave API, reemplace `nnn` en el siguiente comando con su clave.
</div>

In [15]:
!curl -X GET "https://portal.opentopography.org/API/globaldem?demtype=SRTMGL1_E&south=33&north=34&west=-119&east=-118&outputFormat=GTiff&API_Key=fa380d477d946dcf108e55dd98964ea4" -H "accept: */*" --output N33W119_wgs84.tif

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 6948k    0 6948k    0     0   168k      0 --:--:--  0:00:41 --:--:--  364k


Si no logra obtener la clave API de OpenTopography y la descarga de `N33W119_wgs84.tif` no tiene éxito, puede sustituirla por `elevation.dem` en el resto de la sección.

In [16]:
#Lets look at the downloaded Copernicus DEM
# This is a single band file

!gdalinfo elevation.dem

Driver: ROI_PAC/ROI_PAC raster
Files: elevation.dem
       elevation.dem.rsc
Size is 3600, 3600
Coordinate System is:
GEOGCRS["WGS 84",
    DATUM["World Geodetic System 1984",
        ELLIPSOID["WGS 84",6378137,298.257223563,
            LENGTHUNIT["metre",1]]],
    PRIMEM["Greenwich",0,
        ANGLEUNIT["degree",0.0174532925199433]],
    CS[ellipsoidal,2],
        AXIS["geodetic latitude (Lat)",north,
            ORDER[1],
            ANGLEUNIT["degree",0.0174532925199433]],
        AXIS["geodetic longitude (Lon)",east,
            ORDER[2],
            ANGLEUNIT["degree",0.0174532925199433]],
    ID["EPSG",4326]]
Data axis to CRS axis mapping: 2,1
Origin = (-119.000000000000000,35.000000000000000)
Pixel Size = (0.000277777777000,-0.000277777777000)
Corner Coordinates:
Upper Left  (-119.0000000,  35.0000000) (119d 0' 0.00"W, 35d 0' 0.00"N)
Lower Left  (-119.0000000,  34.0000000) (119d 0' 0.00"W, 34d 0' 0.00"N)
Upper Right (-118.0000000,  35.0000000) (118d 0' 0.00"W, 35d 0' 0.00"N)
Lo

In [17]:
#Lets look at the California DEM from UCSB
# This is a single band file

!gdalinfo 18.grd

Driver: GMT/GMT NetCDF Grid Format
Files: 18.grd
Size is 1201, 601
Origin = (-121.002499999999998,35.502499999999998)
Pixel Size = (0.005000000000000,-0.005000000000000)
Corner Coordinates:
Upper Left  (-121.0025000,  35.5025000) 
Lower Left  (-121.0025000,  32.4975000) 
Upper Right (-114.9975000,  35.5025000) 
Lower Right (-114.9975000,  32.4975000) 
Center      (-118.0000000,  34.0000000) 
Band 1 Block=1201x1 Type=Float32, ColorInterp=Undefined


In [19]:
#Lets look at a coherence product generated by ISCE
# This is a two band file

!gdalinfo stripmap/interferogram/topophase.cor.geo.vrt

Driver: VRT/Virtual Raster
Files: stripmap/interferogram/topophase.cor.geo.vrt
       stripmap/interferogram/topophase.cor.geo
Size is 2989, 3547
Coordinate System is:
GEOGCRS["WGS 84",
    ENSEMBLE["World Geodetic System 1984 ensemble",
        MEMBER["World Geodetic System 1984 (Transit)"],
        MEMBER["World Geodetic System 1984 (G730)"],
        MEMBER["World Geodetic System 1984 (G873)"],
        MEMBER["World Geodetic System 1984 (G1150)"],
        MEMBER["World Geodetic System 1984 (G1674)"],
        MEMBER["World Geodetic System 1984 (G1762)"],
        ELLIPSOID["WGS 84",6378137,298.257223563,
            LENGTHUNIT["metre",1]],
        ENSEMBLEACCURACY[2.0]],
    PRIMEM["Greenwich",0,
        ANGLEUNIT["degree",0.0174532925199433]],
    CS[ellipsoidal,2],
        AXIS["geodetic latitude (Lat)",north,
            ORDER[1],
            ANGLEUNIT["degree",0.0174532925199433]],
        AXIS["geodetic longitude (Lon)",east,
            ORDER[2],
            ANGLEUNIT["degree",0.

In [20]:
### ¿Cómo obtenemos esta información programáticamente?
ds = gdal.Open('stripmap/interferogram/topophase.cor.geo.vrt', gdal.GA_ReadOnly)
print('Bands = ', ds.RasterCount)
print('Dimensions = ', ds.RasterXSize, ' x ', ds.RasterYSize)
ds = None

Bands =  2
Dimensions =  2989  x  3547


### Tipos de datos

Los siguientes tipos de datos son compatibles con GDAL:

1. byte
2. UInt16
3. Int16
4. UInt32
5. Int32
6. Flotador32
7. Flotador64
8. **CInt16**
9. **CInt32**
10. **CFloat32**
11. **CFloat64**

El hecho de que GDAL incluya soporte para números complejos lo hace significativamente más relevante que otros sistemas GIS para usar con conjuntos de datos de radar.

In [21]:
!gdalinfo stripmap/interferogram/topophase.flat.vrt

Driver: VRT/Virtual Raster
Files: stripmap/interferogram/topophase.flat.vrt
       stripmap/interferogram/topophase.flat
Size is 2781, 3456
Corner Coordinates:
Upper Left  (    0.0,    0.0)
Lower Left  (    0.0, 3456.0)
Upper Right ( 2781.0,    0.0)
Lower Right ( 2781.0, 3456.0)
Center      ( 1390.5, 1728.0)
Band 1 Block=2781x1 Type=CFloat32, ColorInterp=Undefined


In [22]:
## Método programático de consulta de tipo de datos
ds = gdal.Open('18.grd', gdal.GA_ReadOnly)
band = ds.GetRasterBand(1)
print(gdal.GetDataTypeName(band.DataType))
band = None
ds = None

Float32


### Sistema de coordinadas

Aquí es donde GDAL realmente se destaca. Existen numerosas convenciones que se utilizan globalmente para representar el sistema de coordenadas de los datos de mapas. Aquí hay algunos populares:

1. PROJ4 - https://proj4.org/usage/projections.html
2. USGS - Colección de 18 números, extremadamente inutilizable
3. OGC WKT (estándar de facto) - http://docs.opengeospatial.org/is/12-063r5/12-063r5.html
4. Códigos EPSG (fáciles de usar) - http://spatialreference.org/ref/epsg/
5. XML, etc.

Usaremos **gdalsrsinfo** para demostrar las complicaciones involucradas en el soporte de estos sistemas de coordenadas.

In [23]:
##La proyección latitud/longitud WGS84 simple tiene el código EPSG 4326.
##Para ver todas las representaciones equivalentes de este sistema
!gdalsrsinfo -o all EPSG:4326


PROJ.4 : +proj=longlat +datum=WGS84 +no_defs

OGC WKT1 :
GEOGCS["WGS 84",
    DATUM["WGS_1984",
        SPHEROID["WGS 84",6378137,298.257223563,
            AUTHORITY["EPSG","7030"]],
        AUTHORITY["EPSG","6326"]],
    PRIMEM["Greenwich",0,
        AUTHORITY["EPSG","8901"]],
    UNIT["degree",0.0174532925199433,
        AUTHORITY["EPSG","9122"]],
    AXIS["Latitude",NORTH],
    AXIS["Longitude",EAST],
    AUTHORITY["EPSG","4326"]]

OGC WKT2:2015 :
GEODCRS["WGS 84",
    DATUM["World Geodetic System 1984",
        ELLIPSOID["WGS 84",6378137,298.257223563,
            LENGTHUNIT["metre",1]]],
    PRIMEM["Greenwich",0,
        ANGLEUNIT["degree",0.0174532925199433]],
    CS[ellipsoidal,2],
        AXIS["geodetic latitude (Lat)",north,
            ORDER[1],
            ANGLEUNIT["degree",0.0174532925199433]],
        AXIS["geodetic longitude (Lon)",east,
            ORDER[2],
            ANGLEUNIT["degree",0.0174532925199433]],
    SCOPE["Horizontal component of 3D system."],
    AREA[

In [24]:
## Veamos la zona UTM 20N
!gdalsrsinfo -o all EPSG:32620


PROJ.4 : +proj=utm +zone=20 +datum=WGS84 +units=m +no_defs

OGC WKT1 :
PROJCS["WGS 84 / UTM zone 20N",
    GEOGCS["WGS 84",
        DATUM["WGS_1984",
            SPHEROID["WGS 84",6378137,298.257223563,
                AUTHORITY["EPSG","7030"]],
            AUTHORITY["EPSG","6326"]],
        PRIMEM["Greenwich",0,
            AUTHORITY["EPSG","8901"]],
        UNIT["degree",0.0174532925199433,
            AUTHORITY["EPSG","9122"]],
        AUTHORITY["EPSG","4326"]],
    PROJECTION["Transverse_Mercator"],
    PARAMETER["latitude_of_origin",0],
    PARAMETER["central_meridian",-63],
    PARAMETER["scale_factor",0.9996],
    PARAMETER["false_easting",500000],
    PARAMETER["false_northing",0],
    UNIT["metre",1,
        AUTHORITY["EPSG","9001"]],
    AXIS["Easting",EAST],
    AXIS["Northing",NORTH],
    AUTHORITY["EPSG","32620"]]

OGC WKT2:2015 :
PROJCRS["WGS 84 / UTM zone 20N",
    BASEGEODCRS["WGS 84",
        DATUM["World Geodetic System 1984",
            ELLIPSOID["WGS 84",6378137,2

También puede usar **gdalsrsinfo** para consultar el sistema de coordenadas de un producto ráster dado en un formato específico.

In [25]:
##Obtener información de proyección para un archivo ráster
!gdalsrsinfo -o proj4  N33W119_wgs84.tif


+proj=longlat +datum=WGS84 +no_defs



In [26]:
## Obtener información de proyección para otro archivo ráster que tenga la información de proyección
!gdalsrsinfo -o proj4 elevation.dem


+proj=longlat +datum=WGS84 +no_defs



In [27]:
###Método programático de consulta de referencia espacial
ds = gdal.Open('N33W119_wgs84.tif', gdal.GA_ReadOnly)
srs = ds.GetProjectionRef()
ds = None
print(srs)

ref = osr.SpatialReference()
ref.ImportFromWkt(srs)
ref.ExportToProj4()

GEOGCS["WGS 84",DATUM["WGS_1984",SPHEROID["WGS 84",6378137,298.257223563,AUTHORITY["EPSG","7030"]],AUTHORITY["EPSG","6326"]],PRIMEM["Greenwich",0],UNIT["degree",0.0174532925199433,AUTHORITY["EPSG","9122"]],AXIS["Latitude",NORTH],AXIS["Longitude",EAST],AUTHORITY["EPSG","4326"]]


'+proj=longlat +datum=WGS84 +no_defs'

Cuando usa **gdalsrsinfo** en un ráster que no tiene la información del sistema de coordenadas, o como suele ser el caso con productos de radar en geometría de radar, debería generar un error.

In [28]:
##Esto debería generar un error.
!gdalsrsinfo 18.grd

ERROR 1: ERROR - failed to load SRS definition from 18.grd


In [29]:
##Esto debería generar un error.
!gdalsrsinfo stripmap/topophase.cor

ERROR 1: ERROR - failed to load SRS definition from stripmap/topophase.cor


### Coordenadas en un mapa

En la subsección anterior, discutimos los sistemas de coordenadas. En esta sección, hablamos sobre cómo GDAL asocia una ubicación de línea/píxel en un ráster con coordenadas de mapa.

GDAL utiliza un sistema afín simple para traducir la ubicación (píxel, línea) a las coordenadas (X, Y) en las coordenadas del mapa. El sistema afín está representado por 6 números de doble precisión conocidos colectivamente como ''GeoTransform''.

**Arriba a la izquierda** la coordenada del mapa de un píxel representado por el número de píxel (P) y el número de línea (L) en la indexación de C/Python viene dada por

$$\begin{equation*}
\left[ \begin{array}{c}
X_{left} \\
Y_{top}
\end{array} \right] = \left[ \begin{array}{c}
G_0 \\
G_3 \end{array}\right] + \left[ \begin{array}{cc}
G_1 & G_2 \\
G_4 & G_5 
\end{array} \right] \cdot \left[ \begin{array}{c}
P \\
L 
\end{array} \right]
\end{equation*}
$$


Esta es la manera exacta en que GDAL interpreta los índices de píxeles
<div>
<p style = 'text-align:center;'>
<img src="Figs/Raster_layout.png" width="300"/> 
</p>
</div>

In [30]:
##Para confirmar la interpretación de la parte superior izquierda, veamos la salida de gdalinfo para el mosaico SRTM nuevamente
!gdalinfo N33W119_wgs84.tif

Driver: GTiff/GeoTIFF
Files: N33W119_wgs84.tif
Size is 3600, 3600
Coordinate System is:
GEOGCRS["WGS 84",
    DATUM["World Geodetic System 1984",
        ELLIPSOID["WGS 84",6378137,298.257223563,
            LENGTHUNIT["metre",1]]],
    PRIMEM["Greenwich",0,
        ANGLEUNIT["degree",0.0174532925199433]],
    CS[ellipsoidal,2],
        AXIS["geodetic latitude (Lat)",north,
            ORDER[1],
            ANGLEUNIT["degree",0.0174532925199433]],
        AXIS["geodetic longitude (Lon)",east,
            ORDER[2],
            ANGLEUNIT["degree",0.0174532925199433]],
    ID["EPSG",4326]]
Data axis to CRS axis mapping: 2,1
Origin = (-119.000138888880798,34.000138888885445)
Pixel Size = (0.000277777777778,-0.000277777777778)
Metadata:
  AREA_OR_POINT=Area
Image Structure Metadata:
  COMPRESSION=LZW
  INTERLEAVE=BAND
  LAYOUT=COG
Corner Coordinates:
Upper Left  (-119.0001389,  34.0001389) (119d 0' 0.50"W, 34d 0' 0.50"N)
Lower Left  (-119.0001389,  33.0001389) (119d 0' 0.50"W, 33d 0' 0.50"N

In [31]:
##Una imagen puede incluir información de coordenadas incluso si falta la información del sistema de coordenadas
##Pero esto es muy desaconsejable

!gdalinfo 18.grd

Driver: GMT/GMT NetCDF Grid Format
Files: 18.grd
Size is 1201, 601
Origin = (-121.002499999999998,35.502499999999998)
Pixel Size = (0.005000000000000,-0.005000000000000)
Corner Coordinates:
Upper Left  (-121.0025000,  35.5025000) 
Lower Left  (-121.0025000,  32.4975000) 
Upper Right (-114.9975000,  35.5025000) 
Lower Right (-114.9975000,  32.4975000) 
Center      (-118.0000000,  34.0000000) 
Band 1 Block=1201x1 Type=Float32, ColorInterp=Undefined


In [32]:
###Método programático para consultar la geotransformación

ds = gdal.Open('N33W119_wgs84.tif', gdal.GA_ReadOnly)
print(ds.GetGeoTransform())
ds = None

(-119.0001388888808, 0.00027777777777781464, 0.0, 34.000138888885445, 0.0, -0.00027777777777781464)


In [33]:
###Método programático para consultar la geotransformación

ds = gdal.Open('elevation.dem', gdal.GA_ReadOnly)
print(ds.GetGeoTransform())
ds = None

(-119.0, 0.000277777777, 0.0, 35.0, 0.0, -0.000277777777)


La mayoría de los formatos de datos geoespaciales prefieren la orientación Norte arriba para almacenar datos de mapas. Por lo tanto, es muy común ver un término $G_5$ negativo en la representación de GeoTransform.

In [34]:
###Para un conjunto de datos sin sistema de coordenadas/información de mapa, es decir, ráster simple
ds = gdal.Open('stripmap/interferogram/topophase.cor.vrt', gdal.GA_ReadOnly)
print(ds.GetGeoTransform())
ds = None

(0.5, 2.0, 0.0, 3.5, 0.0, 8.0)


### Puntos de control de tierra (GCPs)

Los GCPs son otro mecanismo para asociar información de coordenadas geográficas con imágenes ráster. Los GCP representan un mapeo explícito desde una ubicación (píxel, línea) a una ubicación (X,Y) en coordenadas de mapa. Por lo general, estas no son las representaciones más precisas, pero para algunas aplicaciones son más que suficientes. Por ejemplo, las imágenes de radar adquiridas sobre terrenos muy planos u océanos se pueden geocodificar razonablemente bien utilizando GCP. No vamos a utilizar esta característica en nuestros tutoriales, pero se debe tener en cuenta que esta información está disponible con la mayoría de los productos SLC entregados por diferentes agencias.

Los archivos geotiff Sentinel-1 SLC y GRD están todos anotados con GCP. A continuación se muestra un ejemplo de salida de gdalinfo en una imagen Sentinel-1 SLC

```bash
> gdalinfo s1a-iw1-slc-vv-20160408t091355-20160408t091428-010728-01001f-001.tiff
Driver: GTiff/GeoTIFF
Files: s1a-iw1-slc-vv-20160408t091355-20160408t091428-010728-01001f-001.tiff
Size is 20665, 17916
Coordinate System is `'
GCP Projection =
GEOGCS["WGS 84",
    DATUM["WGS_1984",
        SPHEROID["unnamed",6378137,298.2572235604902,
            AUTHORITY["EPSG","4326"]],
        AUTHORITY["EPSG","6326"]],
    PRIMEM["Greenwich",0],
    UNIT["degree",0.0174532925199433],
    AUTHORITY["EPSG","4326"]]
GCP[  0]: Id=1, Info=
          (0,0) -> (130.457799786776,32.2907336141953,478.460773384615)
GCP[  1]: Id=2, Info=
          (1034,0) -> (130.506581179458,32.2986697089719,478.460773384615)
GCP[  2]: Id=3, Info=
          (2068,0) -> (130.554913356166,32.3065128120191,478.460773384615)
GCP[  3]: Id=4, Info=
          (3102,0) -> (130.602812666404,32.3142661230211,478.460773384615)
...
GCP[270]: Id=271, Info=
          (18612,17915) -> (130.826037469855,34.4318305450677,0)
GCP[271]: Id=272, Info=
          (19646,17915) -> (130.869763720912,34.438454503529,0)
GCP[272]: Id=273, Info=
          (20664,17915) -> (130.912557438728,34.4449210788505,0)
Metadata:
  AREA_OR_POINT=Area
  TIFFTAG_DATETIME=2016:04:08 16:04:16
  TIFFTAG_IMAGEDESCRIPTION=Sentinel-1A IW SLC L1
  TIFFTAG_SOFTWARE=Sentinel-1 IPF 002.62
Image Structure Metadata:
  INTERLEAVE=BAND
Corner Coordinates:
Upper Left  (    0.0,    0.0)
Lower Left  (    0.0,17916.0)
Upper Right (20665.0,    0.0)
Lower Right (20665.0,17916.0)
Center      (10332.5, 8958.0)
Band 1 Block=20665x1 Type=CInt16, ColorInterp=Gray
```

### Subconjuntos de datos

Hay algunos formatos de archivo como netcdf o HDF5 que permiten empaquetar varios conjuntos de datos. En tales casos, GDAL informará estas diferentes capas como subconjuntos de datos. Exploraremos esto con el conjunto de datos de Groenlandia descargado.

In [35]:
!gdalinfo Greenland1km.nc

Driver: netCDF/Network Common Data Format
Files: Greenland1km.nc
Size is 512, 512
Metadata:
  NC_GLOBAL#comments=The data in this file was interpolated onto a 1km grid using natural neighbor interpolation which is based on Delauney triangulation.  This was accomplished using the function natgridd (see http://www.ncarg.ucar.edu//ngmath/natgrid/nnhome.html) from the National Center for Atmospheric Research (NCAR).  Natgrid is part of the NCAR Command Language (NCL) package which can be downloaded via http://www.ncl.ucar.edu/Download/
  NC_GLOBAL#creator=Glen D. Granzow
  NC_GLOBAL#Date=1 November 2010
  NC_GLOBAL#institution=University of Montana
  NC_GLOBAL#title=Greenland 1 km Data Set
Subdatasets:
  SUBDATASET_1_NAME=NETCDF:"Greenland1km.nc":presprcp
  SUBDATASET_1_DESC=[1x2801x1501] lwe_precipitation_rate (32-bit floating-point)
  SUBDATASET_2_NAME=NETCDF:"Greenland1km.nc":bheatflx
  SUBDATASET_2_DESC=[1x2801x1501] upward_geothermal_flux_at_ice_base (32-bit floating-point)
  SUBDATAS

Para acceder a un subconjunto de datos específico dentro del archivo netcdf, puede usar el atributo "SUBDATASET\_\*\_NAME" informado por gdalinfo.

In [36]:
!gdalinfo NETCDF:"Greenland1km.nc":topg

Driver: netCDF/Network Common Data Format
Files: Greenland1km.nc
Size is 1501, 2801
Coordinate System is:
PROJCRS["unnamed",
    BASEGEOGCRS["WGS 84",
        DATUM["World Geodetic System 1984",
            ELLIPSOID["WGS 84",6378137,298.257223563,
                LENGTHUNIT["metre",1]]],
        PRIMEM["Greenwich",0,
            ANGLEUNIT["degree",0.0174532925199433]],
        ID["EPSG",4326]],
    CONVERSION["unnamed",
        METHOD["Polar Stereographic (variant B)",
            ID["EPSG",9829]],
        PARAMETER["Latitude of standard parallel",71,
            ANGLEUNIT["degree",0.0174532925199433],
            ID["EPSG",8832]],
        PARAMETER["Longitude of origin",-39,
            ANGLEUNIT["degree",0.0174532925199433],
            ID["EPSG",8833]],
        PARAMETER["False easting",0,
            LENGTHUNIT["unknown",1],
            ID["EPSG",8806]],
        PARAMETER["False northing",0,
            LENGTHUNIT["unknown",1],
            ID["EPSG",8807]]],
    CS[Cartesian,2],
 

In [37]:
!gdalsrsinfo NETCDF:"Greenland1km.nc":topg


PROJ.4 : +proj=stere +lat_0=90 +lat_ts=71 +lon_0=-39 +x_0=0 +y_0=0 +datum=WGS84 +units=m +no_defs

OGC WKT2:2018 :
PROJCRS["unnamed",
    BASEGEOGCRS["WGS 84",
        DATUM["World Geodetic System 1984",
            ELLIPSOID["WGS 84",6378137,298.257223563,
                LENGTHUNIT["metre",1]]],
        PRIMEM["Greenwich",0,
            ANGLEUNIT["degree",0.0174532925199433]],
        ID["EPSG",4326]],
    CONVERSION["unnamed",
        METHOD["Polar Stereographic (variant B)",
            ID["EPSG",9829]],
        PARAMETER["Latitude of standard parallel",71,
            ANGLEUNIT["degree",0.0174532925199433],
            ID["EPSG",8832]],
        PARAMETER["Longitude of origin",-39,
            ANGLEUNIT["degree",0.0174532925199433],
            ID["EPSG",8833]],
        PARAMETER["False easting",0,
            LENGTHUNIT["unknown",1],
            ID["EPSG",8806]],
        PARAMETER["False northing",0,
            LENGTHUNIT["unknown",1],
            ID["EPSG",8807]]],
    CS[Carte

### Otras características

Hay otras funciones útiles que proporciona GDAL para trabajar con datos ráster. Uno puede leer más sobre esto aquí:
http://www.gdal.org/gdal_datamodel.html

## Formatos de datos ráster

Ya hemos mencionado que GDAL puede admitir más de 80 formatos de datos. GDAL intenta proporcionar una interfaz unificada para leer datos de estos diferentes formatos. La compatibilidad con diferentes formatos depende de qué bibliotecas se usaron al crear la biblioteca GDAL. Puede averiguar los diversos formatos de trama que admite GDAL usando **gdalinfo**.

In [38]:
#Enumere todos los formatos admitidos en su compilación GDAL
!gdalinfo --formats

Supported Formats:
  VRT -raster,multidimensional raster- (rw+v): Virtual Raster
  DERIVED -raster- (ro): Derived datasets using VRT pixel functions
  GTiff -raster- (rw+vs): GeoTIFF
  COG -raster- (wv): Cloud optimized GeoTIFF generator
  NITF -raster- (rw+vs): National Imagery Transmission Format
  RPFTOC -raster- (rovs): Raster Product Format TOC format
  ECRGTOC -raster- (rovs): ECRG TOC format
  HFA -raster- (rw+v): Erdas Imagine Images (.img)
  SAR_CEOS -raster- (rov): CEOS SAR Image
  CEOS -raster- (rov): CEOS Image
  JAXAPALSAR -raster- (rov): JAXA PALSAR Product Reader (Level 1.1/1.5)
  GFF -raster- (rov): Ground-based SAR Applications Testbed File Format (.gff)
  ELAS -raster- (rw+v): ELAS
  ESRIC -raster- (rov): Esri Compact Cache
  AIG -raster- (rov): Arc/Info Binary Grid
  AAIGrid -raster- (rwv): Arc/Info ASCII Grid
  GRASSASCIIGrid -raster- (rov): GRASS ASCII Grid
  ISG -raster- (rov): International Service for the Geoid
  SDTS -raster- (rov): SDTS Raster
  DTED -raster- 

La capacidad para cada formato se indica con una cadena en la salida y aquí está la leyenda correspondiente:

1. r: leer
2. ro: solo lectura
3. w: escriba "Crear conjunto de datos copiando otro"
4. w+: escribir con soporte para actualizar "crear conjunto de datos grabable"
5. s: admite subconjuntos de datos
6. v: admite E/S virtual, p. /vsimem/

Las características virtuales son especiales y las revisaremos más adelante en este tutorial.

Para demostrar que se pueden usar diferentes formatos para representar los mismos datos, hemos incluido una carpeta llamada **DEM** que contiene el DEM SRTM 90m para el mismo mosaico en 3 formatos diferentes: formato hgt nativo SRTM (de LPDAAC), GMT formato (del servicio dem de GMTSAR) y formato Geotiff (OpenTopography). Puede verificar que el contenido sea el mismo ejecutando el comando.

In [39]:
!gdalinfo -stats N33W119_wgs84.tif

Driver: GTiff/GeoTIFF
Files: N33W119_wgs84.tif
Size is 3600, 3600
Coordinate System is:
GEOGCRS["WGS 84",
    DATUM["World Geodetic System 1984",
        ELLIPSOID["WGS 84",6378137,298.257223563,
            LENGTHUNIT["metre",1]]],
    PRIMEM["Greenwich",0,
        ANGLEUNIT["degree",0.0174532925199433]],
    CS[ellipsoidal,2],
        AXIS["geodetic latitude (Lat)",north,
            ORDER[1],
            ANGLEUNIT["degree",0.0174532925199433]],
        AXIS["geodetic longitude (Lon)",east,
            ORDER[2],
            ANGLEUNIT["degree",0.0174532925199433]],
    ID["EPSG",4326]]
Data axis to CRS axis mapping: 2,1
Origin = (-119.000138888880798,34.000138888885445)
Pixel Size = (0.000277777777778,-0.000277777777778)
Metadata:
  AREA_OR_POINT=Area
Image Structure Metadata:
  COMPRESSION=LZW
  INTERLEAVE=BAND
  LAYOUT=COG
Corner Coordinates:
Upper Left  (-119.0001389,  34.0001389) (119d 0' 0.50"W, 34d 0' 0.50"N)
Lower Left  (-119.0001389,  33.0001389) (119d 0' 0.50"W, 33d 0' 0.50"N

In [40]:
!gdalinfo -stats 18.grd

Driver: GMT/GMT NetCDF Grid Format
Files: 18.grd
Size is 1201, 601
Origin = (-121.002499999999998,35.502499999999998)
Pixel Size = (0.005000000000000,-0.005000000000000)
Corner Coordinates:
Upper Left  (-121.0025000,  35.5025000) 
Lower Left  (-121.0025000,  32.4975000) 
Upper Right (-114.9975000,  35.5025000) 
Lower Right (-114.9975000,  32.4975000) 
Center      (-118.0000000,  34.0000000) 
Band 1 Block=1201x1 Type=Float32, ColorInterp=Undefined
  Minimum=-74.586, Maximum=3411.414, Mean=443.527, StdDev=504.287
  Metadata:
    STATISTICS_MAXIMUM=3411.4138183594
    STATISTICS_MEAN=443.52650734192
    STATISTICS_MINIMUM=-74.586204528809
    STATISTICS_STDDEV=504.28678716346
    STATISTICS_VALID_PERCENT=100


<br>
<div class="alert alert-info">
<b>Nota:</b>
Como puede ver, GDAL admite una amplia gama de formatos de datos y proporciona una interfaz unificada. Si planea trabajar con conjuntos de datos ráster, le recomendamos que utilice uno de estos formatos admitidos como base. Tiene que haber una razón realmente convincente para elegir un formato diferente.
</div>



## Sistemas de archivos virtuales

GDAL no necesariamente necesita que los datos se almacenen en un archivo local. Proporciona una interfaz similar a un archivo para varias fuentes de datos y estos se conocen como formatos virtuales. Estos sistemas se denominan sistemas de archivos virtuales. Para obtener una lista completa de los sistemas de archivos virtuales admitidos por GDAL, consulte http://www.gdal.org/gdal_virtual_file_systems.html

Veremos un par de ejemplos de cómo funcionan.

### vsicurl

Esto está destinado a ser utilizado con conjuntos de datos que se sirven en línea a través de http/https/ftp. Para habilitar esta función, GDAL debe construirse con soporte "curl". Aquí hay un ejemplo de cómo usaríamos esta característica

In [41]:
#Este es un ejemplo de trabajo con el archivo Landsat de Amazon
!gdalinfo /vsicurl/http://landsat-pds.s3.amazonaws.com/L8/001/003/LC80010032014272LGN00/LC80010032014272LGN00_B1.TIF    

ERROR 11: HTTP response code: 404
gdalinfo failed - unable to open '/vsicurl/http://landsat-pds.s3.amazonaws.com/L8/001/003/LC80010032014272LGN00/LC80010032014272LGN00_B1.TIF'.


### vsizip / vsitar / vsigzip

In [43]:
#Mire el DEM descargado del sitio de PSU
!zipinfo DEM.zip

Archive:  DEM.zip
Zip file size: 2600801 bytes, number of entries: 12
drwx---     2.0 fat        0 b- stor 04-Nov-17 10:01 DEM_10m/
-rw-a--     2.0 fat  9687040 t- defN 03-Mar-18 06:39 DEM_10m/bushkill_pa.dem
-rw-a--     2.0 fat    29376 t- defN 03-Apr-05 18:41 DEM_10m/bushkill_pa.html
-rw-a--     2.0 fat    23087 t- defN 03-Apr-05 18:41 DEM_10m/bushkill_pa.text
-rw-a--     2.0 fat      973 t- defN 03-Feb-07 15:48 DEM_10m/bushkill_pa.txt
-rw-a--     2.0 fat    20758 t- defN 03-Apr-05 18:41 DEM_10m/bushkill_pa.xml
drwx---     2.0 fat        0 b- stor 04-Nov-17 10:01 DEM_30m/
-rw-a--     2.0 fat    26279 t- defN 03-Aug-28 08:11 DEM_30m/bushkill_dem_30m_metadata.html
drwx---     2.0 fat        0 b- stor 04-Nov-17 10:01 DEM_30m/bushkill_dem_30m_metadata_files/
-rw-a--     2.0 fat      104 b- defN 03-Aug-28 08:10 DEM_30m/bushkill_dem_30m_metadata_files/disk.gif
-rw-ah-     2.0 fat     3072 b- defN 03-Aug-28 08:33 DEM_30m/bushkill_dem_30m_metadata_files/Thumbs.db
-rw-a--     2.0 fat  1077248

In [44]:
###Interactuar directamente con datos dentro de archivos zip/tar
!gdalinfo /vsizip/DEM.zip/DEM_30m/bushkill_pa.dem

Driver: USGSDEM/USGS Optional ASCII DEM (and CDED)
Files: /vsizip/DEM.zip/DEM_30m/bushkill_pa.dem
Size is 350, 465
Coordinate System is:
PROJCRS["unnamed",
    BASEGEOGCRS["NAD27",
        DATUM["North American Datum 1927",
            ELLIPSOID["Clarke 1866",6378206.4,294.978698213898,
                LENGTHUNIT["metre",1]]],
        PRIMEM["Greenwich",0,
            ANGLEUNIT["degree",0.0174532925199433]],
        ID["EPSG",4267]],
    CONVERSION["UTM zone 18N",
        METHOD["Transverse Mercator",
            ID["EPSG",9807]],
        PARAMETER["Latitude of natural origin",0,
            ANGLEUNIT["degree",0.0174532925199433],
            ID["EPSG",8801]],
        PARAMETER["Longitude of natural origin",-75,
            ANGLEUNIT["degree",0.0174532925199433],
            ID["EPSG",8802]],
        PARAMETER["Scale factor at natural origin",0.9996,
            SCALEUNIT["unity",1],
            ID["EPSG",8805]],
        PARAMETER["False easting",500000,
            LENGTHUNIT["metre",

### vsis3/ vsigs/ vsiaz

Muchas de las nuevas actualizaciones en GDAL se enfocan en ampliar el soporte a los datos almacenados en plataformas en la nube, lo que permite a los usuarios interactuar con pequeñas porciones de datos sin tener que descargar todo.

#### Opción 1: Usar vsicurl y cookies con archivo .netrc

En este caso, reutilizaremos la configuración del archivo .netrc que configuramos anteriormente para las descargas automáticas de DEM desde LP DAAC. Si no lo ha hecho, complete el Paso 2 desde el enlace que se muestra aquí:
https://wiki.earthdata.nasa.gov/display/EL/How+To+Access+Data+With+cURL+And+Wget

In [46]:
gdal.SetConfigOption('GDAL_HTTP_COOKIEFILE','asf_cookies.txt')
gdal.SetConfigOption('GDAL_HTTP_COOKIEJAR', 'asf_cookies.txt')
results = gdal.Info("/vsicurl/https://grfn.asf.alaska.edu/door/download/S1-GUNW-D-R-160-tops-20190710_20190628-162436-20935N_18926N-PP-6a53-v2_0_2.nc")
print(results)

None


ERROR 4: `/vsicurl/https://grfn.asf.alaska.edu/door/download/S1-GUNW-D-R-160-tops-20190710_20190628-162436-20935N_18926N-PP-6a53-v2_0_2.nc' does not exist in the file system, and is not recognized as a supported dataset name.


Más información sobre cómo interactuar con los datos que ASF aloja en Amazon Cloud en tutoriales posteriores...

### vsimem

Estas son características que permiten su uso en matrices de memoria al igual que otros conjuntos de datos GDAL. Aquí hay un ejemplo:

In [47]:
##Get memory driver
memdrv = gdal.GetDriverByName('MEM')

##Open source dataset in read only mode
ds = gdal.Open('N33W119_wgs84.tif', gdal.GA_ReadOnly)

##Create an in-memory copy
memds = memdrv.CreateCopy('dummyname', ds)

##Close source
ds = None

##Use copy for operations
print(memds.GetProjection())
print(memds.GetGeoTransform())

##Close memory copy
memds = None

GEOGCS["WGS 84",DATUM["WGS_1984",SPHEROID["WGS 84",6378137,298.257223563,AUTHORITY["EPSG","7030"]],AUTHORITY["EPSG","6326"]],PRIMEM["Greenwich",0],UNIT["degree",0.0174532925199433,AUTHORITY["EPSG","9122"]],AXIS["Latitude",NORTH],AXIS["Longitude",EAST],AUTHORITY["EPSG","4326"]]
(-119.0001388888808, 0.00027777777777781464, 0.0, 34.000138888885445, 0.0, -0.00027777777777781464)


## Formato Virtual Raster Table (VRT)

El formato de tabla ráster virtual (VRT) amplía la idea de virtualización a discos reales en archivo. VRT es un formato personalizado compatible con GDAL que es similar a los "enlaces" o "accesos directos" en los sistemas operativos, excepto que tiene más funciones que simplemente apuntar a otras fuentes de datos.

A menudo, cuando trabajamos con conjuntos de datos ráster, tendemos a generar un volumen bastante grande de productos intermedios debido a operaciones como recortar, mirar múltiples y enmascarar. Los VRT ofrecen la opción de crear canales de flujo de trabajo que generan dichos productos sobre la marcha. Nos basaremos ampliamente en esta función para nuestros ejercicios de series de tiempo más adelante.

Los detalles sobre la función VRT de GDAL se pueden encontrar aquí: http://www.gdal.org/gdal_vrttut.html

Es posible que ya haya notado que ISCE genera un archivo **.vrt** para cada producto ráster generado por los flujos de trabajo. Veremos los VRT con mayor detalle como parte de los otros cuadernos sobre manipulación de datos ráster. Pero aquí hay algunos ejemplos breves que demuestran las capacidades de este formato.

### Recorte de conjuntos de datos

ISCE utiliza archivos GDAL VRT para extraer virtualmente una sola ráfaga de los productos Sentinel-1 SAFE. De nuestro tutorial de procesamiento TOPS, recordará que los datos del modo TOPS de Sentinel-1 se entregan como productos SEGUROS que contienen archivos TIFF para cada franja de imágenes. Cada franja consiste en un número de ráfagas o escaneos.

Para evitar replicar estos datos en el disco nuevamente, usamos un archivo VRT simple para señalar la ubicación de los datos de ráfaga dentro de los archivos TIFF. Por ejemplo, en la salida de procesamiento de topsApp.ipynb, habrá SLC de ráfaga procesados organizados en carpetas, donde "referencia" es la imagen de referencia de un interferograma y "secundaria" es la imagen que se interferirá con la referencia. Si tuviera que mirar el contenido de reference/IW1/burst04.slc.vrt, verá algo como

```xml
<VRTDataset rasterXSize="20665" rasterYSize="1493">
    <VRTRasterBand dataType="CFloat32" band="1">
        <NoDataValue>0.0</NoDataValue>
        <SimpleSource>
            <SourceFilename relativeToVRT="1">/vsizip//Users/agram/data/isce/japan/data/S1A_IW_SLC__1SSV_20160408T091355_20160408T091430_010728_01001F_83EB.zip/S1A_IW_SLC__1SSV_20160408T091355_20160408T091430_010728_01001F_83EB.SAFE/measurement/s1a-iw1-slc-vv-20160408t091355-20160408t091428-010728-01001f-001.tiff</SourceFilename>
            <SourceBand>1</SourceBand>
            <SourceProperties RasterXSize="20665" RasterYSize="17916" DataType="CInt16"/>
            <SrcRect xOff="130" yOff="4498" xSize="20468" ySize="1458"/>
            <DstRect xOff="130" yOff="19" xSize="20468" ySize="1458"/>
        </SimpleSource>
    </VRTRasterBand>
</VRTDataset>
```

Para mayor claridad, anotemos el archivo vrt para describir lo que está sucediendo. Tenga en cuenta que el formato VRT se basa en XML. Por lo tanto, utilizaremos la función de comentarios de XML para anotar el XML para su comprensión.

```xml
<!--Size of the virtual dataset-->
<VRTDataset rasterXSize="20665" rasterYSize="1493">
    <!-- Band 1 of the dataset and its data type -->
    <VRTRasterBand dataType="CFloat32" band="1">
        <!--No data value for this dataset-->
        <NoDataValue>0.0</NoDataValue>
        <!--We use a simple source to suggest that we don't alter contents of the data in source-->
        <SimpleSource>
            <!--Name of the source - notice that we point to a TIFF inside a zip file -->
            <SourceFilename relativeToVRT="1">/vsizip//Users/agram/data/isce/japan/data/S1A_IW_SLC__1SSV_20160408T091355_20160408T091430_010728_01001F_83EB.zip/S1A_IW_SLC__1SSV_20160408T091355_20160408T091430_010728_01001F_83EB.SAFE/measurement/s1a-iw1-slc-vv-20160408t091355-20160408t091428-010728-01001f-001.tiff</SourceFilename>
            <!--Source may contain multiple bands. Indicate band from source -->
            <SourceBand>1</SourceBand>
            <!--Properties of the source. This is optional. GDAL will figure it out on the fly if not provided, but will be slow as it actually has to read the source to figure this out. Note that source is of type CInt16 -->
            <SourceProperties RasterXSize="20665" RasterYSize="17916" DataType="CInt16"/>
            <!--Indicate spatial subset of source data in pixels and lines -->
            <SrcRect xOff="130" yOff="4498" xSize="20468" ySize="1458"/>
            <!--Indicate spatial location of destination-->
            <DstRect xOff="130" yOff="19" xSize="20468" ySize="1458"/>
        </SimpleSource>
    </VRTRasterBand>
</VRTDataset>
```

### Modificar algunos metadatos

¿Recuerda el conjunto de datos en formato GMT del ejercicio anterior? Recuerde que fue etiquetado con información de coordenadas espaciales pero no con metadatos del sistema de coordenadas. Podemos arreglar eso rápidamente escribiendo un VRT que apunte al archivo con la información de coordenadas incluida en él.

In [48]:
!gdal_translate -of VRT -a_srs EPSG:4326 18.grd UCSB_18arcsec.dem.vrt

Input file size is 1201, 601


In [49]:
!cat UCSB_18arcsec.dem.vrt

<VRTDataset rasterXSize="1201" rasterYSize="601">
  <SRS dataAxisToSRSAxisMapping="2,1">GEOGCS["WGS 84",DATUM["WGS_1984",SPHEROID["WGS 84",6378137,298.257223563,AUTHORITY["EPSG","7030"]],AUTHORITY["EPSG","6326"]],PRIMEM["Greenwich",0,AUTHORITY["EPSG","8901"]],UNIT["degree",0.0174532925199433,AUTHORITY["EPSG","9122"]],AXIS["Latitude",NORTH],AXIS["Longitude",EAST],AUTHORITY["EPSG","4326"]]</SRS>
  <GeoTransform> -1.2100250000000000e+02,  5.0000000000000001e-03,  0.0000000000000000e+00,  3.5502499999999998e+01,  0.0000000000000000e+00, -5.0000000000000001e-03</GeoTransform>
  <VRTRasterBand dataType="Float32" band="1" blockYSize="1">
    <Metadata>
      <MDI key="STATISTICS_MAXIMUM">3411.4138183594</MDI>
      <MDI key="STATISTICS_MEAN">443.52650734192</MDI>
      <MDI key="STATISTICS_MINIMUM">-74.586204528809</MDI>
      <MDI key="STATISTICS_STDDEV">504.28678716346</MDI>
      <MDI key="STATISTICS_VALID_PERCENT">100</MDI>
    </Metadata>
    <SimpleSource>
      <SourceFilename relative

In [50]:
!gdalinfo UCSB_18arcsec.dem.vrt

Driver: VRT/Virtual Raster
Files: UCSB_18arcsec.dem.vrt
       18.grd
Size is 1201, 601
Coordinate System is:
GEOGCRS["WGS 84",
    DATUM["World Geodetic System 1984",
        ELLIPSOID["WGS 84",6378137,298.257223563,
            LENGTHUNIT["metre",1]]],
    PRIMEM["Greenwich",0,
        ANGLEUNIT["degree",0.0174532925199433]],
    CS[ellipsoidal,2],
        AXIS["geodetic latitude (Lat)",north,
            ORDER[1],
            ANGLEUNIT["degree",0.0174532925199433]],
        AXIS["geodetic longitude (Lon)",east,
            ORDER[2],
            ANGLEUNIT["degree",0.0174532925199433]],
    ID["EPSG",4326]]
Data axis to CRS axis mapping: 2,1
Origin = (-121.002499999999998,35.502499999999998)
Pixel Size = (0.005000000000000,-0.005000000000000)
Corner Coordinates:
Upper Left  (-121.0025000,  35.5025000) (121d 0' 9.00"W, 35d30' 9.00"N)
Lower Left  (-121.0025000,  32.4975000) (121d 0' 9.00"W, 32d29'51.00"N)
Upper Right (-114.9975000,  35.5025000) (114d59'51.00"W, 35d30' 9.00"N)
Lower Righ

### Conjuntos de datos derivados

A menudo, terminamos en situaciones en las que necesitamos manipular ligeramente los datos para usarlos con otras herramientas de análisis. Por ejemplo, obtenga potencia en dB o amplitud o fase de SLC corregistrados. La generación de estos productos derivados podría consumir mucho espacio en disco. Los VRT proporcionan un mecanismo denominado **funciones de píxeles** para realizar estas transformaciones sobre la marcha. El conjunto de funciones de píxeles disponibles son (lista actual):

* "real": extrae la parte real de una sola banda de trama (solo una copia si la entrada no es compleja)
* "imag": extrae una parte imaginaria de una sola banda de trama (0 para no compleja)
* "complejo": haga una banda compleja fusionando dos bandas utilizadas como valores reales e imag
* "mod": módulo de extracción de una sola banda de trama (real o compleja)
* "fase": extrae la fase de una sola banda de trama [-PI, PI] (0 o PI para no compleja)
* "conj": calcula el conjugado complejo de una sola banda ráster (solo una copia si la entrada no es compleja)
* "suma": suma 2 o más bandas ráster
* "diff": calcula la diferencia entre 2 bandas de trama (b1 - b2)
* "mul": multiplica 2 o más bandas de trama
* "cmul": multiplica la primera banda por el complejo conjugado de la segunda
* "inv": inversa (1./x). Nota: no se realiza verificación en la división cero
* "intensidad": calcula la intensidad Re(x*conj(x)) de una sola banda ráster (real o compleja)
* "sqrt": realiza la raíz cuadrada de una sola banda ráster (solo real)
* "log10": calcula el logaritmo (base 10) de los abs de una sola banda raster (real o compleja): log10( abs( x ) )
* "dB": realiza la conversión a dB de los abs de una sola banda ráster (real o compleja): 20. * log10( abs( x ) )
* "dB2amp": realiza la conversión de escala de logarítmica a lineal (amplitud) (es decir, 10 ^ (x/20)) de una sola banda de trama (solo real)
* "dB2pow": realiza la conversión de escala de logarítmica a lineal (potencia) (es decir, 10 ^ (x/10)) de una sola banda de trama (solo real)


Tenga en cuenta que esta lista es muy relevante para los usuarios de SAR/InSAR ;)

# <u> Ahora está listo para comenzar a explorar los productos de datos geocodificados y no empaquetados de ARIA mediante GDAL. </u>